In [39]:
# Step1: Create OpenVINO Runtime Core
from openvino.runtime import Core
core = Core()

In [40]:
# Step2: Compile the Model, CompiledModel allow you to get information inputs
# or output ports by a tensor name or index
net = core.compile_model("yolov5s.xml","AUTO",{"PERFORMANCE_HINT": "LATENCY"})


In [41]:
# Learn the information of input node
input_node = net.inputs[0]
print(input_node)
print(input_node.shape, input_node.element_type, input_node.index)
print(input_node.names, type(input_node.names), input_node.any_name, type(input_node.any_name))

<ConstOutput: names[images] shape{1,3,640,640} type: f32>
{1, 3, 640, 640} <Type: 'float32'> 0
{'images'} <class 'set'> images <class 'str'>


In [42]:
# Learn the information of output node
output_node = net.outputs[0]
print(output_node)

<ConstOutput: names[output] shape{1,25200,85} type: f32>


In [43]:
# Prepare the input data
import cv2
from utils.augmentations import letterbox
img = cv2.imread("data/images/zidane.jpg")
#img = cv2.imread("data/images/bus.jpg")
print(img.shape)
letterbox_img, ratio, (dw, dh) = letterbox(img, auto=False)
print(letterbox_img.shape, ratio, dw, dh)



(720, 1280, 3)
(640, 640, 3) (0.5, 0.5) 0.0 140.0


In [44]:
# Normalization + Swap RB + Layout from HWC to NCHW
blob = cv2.dnn.blobFromImage(letterbox_img, 1/255.0, swapRB=True)
blob.shape


(1, 3, 640, 640)

In [48]:
# Step 3-5: Do the inference
outs = net({input_node:blob})[output_node]
print(outs.shape)

(1, 25200, 85)


In [49]:
# Step6: Process inference results
import numpy as np
class_ids = []
confidences = []
boxes = []

for out in outs:
    for i, det in enumerate(out):
        confidence = det[4]
        scores = det[5:]
        class_id = np.argmax(scores)
        if scores[class_id] > 0.25:
            confidences.append(confidence)
            class_ids.append(class_id)
            x,y,w,h = det[0].item(),det[1].item(),det[2].item(),det[3].item()
            left = int((x - 0.5*w -dw) / ratio[0])
            top = int((y - 0.5*h - dh) / ratio[1])
            width = int(w / ratio[0])
            height = int(h / ratio[1])
            box = np.array([left, top, width, height])
            boxes.append(box)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45)

filtered_ids = []
filered_confidences = []
filtered_boxes = []

for i in indexes:
    filtered_ids.append(class_ids[i])
    filered_confidences.append(confidences[i])
    filtered_boxes.append(boxes[i])

In [50]:
import yaml
with open("data/coco.yaml","r",encoding="utf-8") as f:
    result = yaml.load(f.read(), Loader=yaml.FullLoader)
class_names = result["names"]

In [51]:
# 调色板
colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0)]
# 显示检测框bbox

for (class_id, confidence, box) in zip(filtered_ids, filered_confidences, filtered_boxes):
    color = colors[int(class_id) % len(colors)]
    cv2.rectangle(img, box, color, 2)
    cv2.rectangle(img, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)
    cv2.putText(img, class_names[class_id], (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    print(class_names[class_id],box)
    
print("Detections: " + str(len(filtered_ids)))
cv2.imshow("YOLOv5+OpenVINO Demo", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

person [743  47 397 670]
tie [442 437  54 272]
person [125 192 585 520]
tie [978 313  46 101]
Detections: 4
